# Lab 7 -- Network Layer (Data Plane)

## Introduction

This lab aims to introduce you to the network layer of the Internet protocol suite. 
The focus is on the data plane, so you will be configuring Network Address Translation (NAT) and Dynamic Host Configuration Protocol (DHCP).

As part of the second delivery submit a PDF report where you include answers from transport and network layer labs and **only** those questions marked with a **REPORT**.

<div class="alert alert-block alert-warning">
  <b>Important:</b> You will be using Graphical Network Simulator-3 (<a href="https://docs.gns3.com/docs/" target="_blank">GNS3</a>). 
  It is an open-source software, widely used among network engineers.
  Ensure that the lab setup is correct before beginning Milestone 1.
</div>

## Lab Setup

[Figure 1](#figure_1) shows the setup for this lab. 
It is an extension to lab2, but the servers now have public IP addresses and two other private networks. 
Your job is to set up NAT to access the servers from the private networks. 
Additionally, you will set up DHCP to dynamically assign IP addresses to clients in private networks.



<a id='figure_1'></a>

|<img src="figures/lab7.png" />|
|:--:| 
| *Figure 1: Lab7 Setup* |


### Lab Topology

- In the ntnu_server (your VM), **build the new container images** using the `create_docker_images.sh` script, before starting this lab.
- From the '~/labs/lab7' directory, use the following command to run the script:
<div style="background: #f8f8f8; overflow:auto;width:auto;border:solid gray;border-width:.1em .1em .1em .8em;padding:.2em .6em;"><pre style="margin: 0; line-height: 125%;color: black;">
./create_docker_images.sh
</pre></div>

- Alternatively, provide the entire path for the shell script as a command, e.g., `~/labs/lab7/create_docker_images.sh`

  <div class="alert alert-block alert-info">
    <b>Note:</b> It will take a while (about 3–5 minutes) to build the docker images.
    Use that time to understand the topology and configure port forwarding (below).
  </div>
  <div class="alert alert-block alert-info">
    <b>Note:</b> Building the container images should only be done once. For the remainder of this lab, as well as the next, you are going to be using these same images. You are able to see the images using the <code style="color: black;">docker images</code> command. <b>GNS3 uses these images to create container instances of the images</b>.
  </div>

### Enable Port Forwarding

- **While building the images**, you need to setup port forwarding to ports which are used by GNS3. Open a new terminal on __*Sahara PC*__, and edit the `~/.ssh/config`:

<div style="background: #f8f8f8; overflow:auto;width:auto;border:solid gray;border-width:.1em .1em .1em .8em;padding:.2em .6em;">
  <pre style="margin: 0; line-height: 125%;color: black;">
Host ntnu_server
  HostName &lt;your_server_IP_address&gt;
  User &lt;your_NTNU_username&gt;
  IdentityFile ~/.ssh/&lt;key_name&gt;
  LocalForward 3080 localhost:3080
  LocalForward 5000 localhost:5000
  LocalForward 5001 localhost:5001
  LocalForward 5002 localhost:5002
  LocalForward 5003 localhost:5003
  LocalForward 5004 localhost:5004
  LocalForward 5005 localhost:5005
  LocalForward 5006 localhost:5006
  LocalForward 5007 localhost:5007
  LocalForward 5008 localhost:5008
  LocalForward 5009 localhost:5009
  LocalForward 5010 localhost:5010
  LocalForward 5011 localhost:5011
  LocalForward 5012 localhost:5012
  LocalForward 5013 localhost:5013
  LocalForward 5014 localhost:5014
  LocalForward 5015 localhost:5015
  LocalForward 5016 localhost:5016
  LocalForward 5017 localhost:5017
  LocalForward 5018 localhost:5018
  LocalForward 5019 localhost:5019
  LocalForward 5020 localhost:5020
  LocalForward 5021 localhost:5021
  LocalForward 5022 localhost:5022
  LocalForward 5023 localhost:5023
  LocalForward 5024 localhost:5024
  LocalForward 5025 localhost:5025
  LocalForward 5026 localhost:5026
  LocalForward 5027 localhost:5027
  LocalForward 5028 localhost:5028
  LocalForward 5029 localhost:5029
  LocalForward 5030 localhost:5030
  LocalForward 5031 localhost:5031
  LocalForward 5032 localhost:5032
  LocalForward 5033 localhost:5033
  LocalForward 5034 localhost:5034</pre>
</div>

- This is just an arbitrary number of ports which GNS3 may use at some point. You may require only a few of them, or you might need even more. In that case, add more lines to your SSH configuration file.

### Update GNS3-Server

- **Check the version of GNS3-Client on Sahara PC**:
  1. Open GNS3 on the Sahara PC.
  2. Locate _"Running GNS3 version x.y.z"_ in the console.
  3. Take note of the version number, i.e. x.y.z from above.

- **Check the version of GNS3-Server on "ntnu_server"**:
  1. In the terminal of "ntnu_server", enter the command `gns3server -v`
  2. Compare with the client version.
  3. If the version numbers are identical, you may skip to the next subsection (Configure GNS3 Client).

- **If the GNS3-Client version is newer than GNS3-Server**:
  1. In the terminal of "ntnu_server", enter the command `sudo apt update && sudo apt install --only-upgrade gns3-server`.
  2. Continue to the next subsection (Configure GNS3 Client).

- **If the GNS3-Server version is newer than GNS3-Client**:
  - To update the GNS3-Client version on Sahara PCs, the TAs will need to contact the IT support. Please inform them if this is the case.
  - After the newest version is installed, you can continue to the next subsection (Configure GNS3 Client).
  - If updating the GNS3-Client is unavailable, you may proceed with the lab on your own computer. Install the newest version of GNS3-Client.
  - Remember to setup `~/.ssh/config` on your computer, as well as a SSH key for more convenient access. Consider using Windows Subsystem for Linux (<a href="https://learn.microsoft.com/en-us/windows/wsl/install" target="_blank">WSL</a>) if running a Windows computer.
  - You may instead attempt to downgrade the GNS3-Server version <i>at your own risk</i>.

### Configure GNS3 Client

- **Open GNS3 (can be found in Applications) on Sahara PC and configure your local GNS3 installation**:
  1. Locate: _Edit -> Preferences -> Server (Or Controller)_
  2. Untick _Enable local server_
  3. Protocol: _HTTP_
  4. Host: _localhost_
  5. Port: _3080 TCP_
  6. Untick _Auth_

- **After building the images**, reconnect to your ntnu_server, such that the open ports take effect. 
Make sure that GNS3 connects to your VM under _Servers Summary_. 

- Import the `lab7.gns3project` as a portable project in GNS3 to view the topology, access the containers, and start packet captures. You can do that in _File -> Import portable project_.

- Start all the devices in GNS3 with "Start/Resume all nodes" button.

- You can access a terminal of a container from GNS3 by double clicking the container.

  <div class="alert alert-block alert-info">
  <b>Tip:</b> To copy files between the local machine (Sahara PC) and the "ntnu_server", you can use the <code style="color: black;"> scp &lt;server_ip_or_name&gt;:&lt;remote_file_path&gt; &lt;local_path&gt;</code> command on your local machine. E.g. <code style="color: black;">scp ntnu_server:~/labs/lab7/lab7.gns3project ~/.</code> to download the GNS3 project file.
  </div>

  <div class="alert alert-block alert-info">
  <b>Tip:</b> To copy files between the "ntnu_server" and the containers, you can use the <code style="color: black;">docker cp &lt;container_name&gt;:&lt;file_path&gt; &lt;local_path&gt;</code> command in "ntnu_server".
  </div>

  <div class="alert alert-block alert-info">
  <b>Tip 3:</b> In order to save certain configurations on the containers when using GNS3, you <b>need</b> to run the save script provided in the home directory (<b>~</b>). Use <code style="color: black;">./save.sh</code> to run it. This will save any <b>nft</b> and <b>vtysh</b> configuration. If you relaunch the GNS3 project without saving, these configurations <b>will be lost</b>. Have a look at the script to understand how to save such configurations.
  </div>

- **The servers** are already configured, so you don't need to do anything with them. 

- You can still use `docker attach` along with the generated name for the container (found with `docker ps`) in your VM in order to access the containers, as long as the topology is started in GNS3. If you want to detach, press `Ctrl+p` followed by `Ctrl+q`, as usual.

### Setup routers

- In the ntnu_server (your VM) **configure the routers** using the `router_setup.py` script. 
**The routers** will then be configured with static routes.

  <div class="alert alert-block alert-info">
  <b>Note:</b> You will have to run this script after each time you launch (i.e. press the green play button) in this GNS3 project for the remainder of <b>lab7</b>. The static routes are not persistent across sessions.
  </div>


In [ ]:
from test_lab7 import TestLab7
check_progress = TestLab7()

# Milestone 1 -- Network Address Translation (NAT)

* In __"router0"__, bring down the interface "ether2".

<div style="background: #f8f8f8; overflow:auto;width:auto;border:solid gray;border-width:.1em .1em .1em .8em;padding:.2em .6em;"><pre style="margin: 0; line-height: 125%;color: black;">
sudo ip link set ether2 down
</pre></div>


* Connect to the webserver from __"router1"__, and retrieve the HTML page:

<div style="background: #f8f8f8; overflow:auto;width:auto;border:solid gray;border-width:.1em .1em .1em .8em;padding:.2em .6em;"><pre style="margin: 0; line-height: 125%;color: black;">
wget http://129.100.1.3 --header "Host: www.ttm4200.com"
</pre></div>


* However, you can not retrieve the webserver HTML page from  __"pclient1"__.

 <span style="font-size: 25px;">Q1. </span> **REPORT:** Briefly explain why you cannot connect to the webserver from  __"pclient1"__. 
 Find out where the packets get dropped and support your claim with screenshots from tcpdump.

<div class="alert alert-block alert-info">
<b>Hint:</b> You can use <code style="color: black;">tcpdump</code> to print packets directly on a network interface in the terminal. 
For example, you can run this command <code style="color: black;">sudo tcpdump port 80 -i [interface_name] --immediate-mode -nn</code> on different routers to check if packets arrive at that interface (refer back to <a href="#figure_1">Figure 1</a>). Then, you can enter the http get command in "pclient1": <code style="color: black;">wget http://129.100.1.3 --header "Host: www.ttm4200.com"</code>.

There can be a delay between starting <code style="color: black;">tcpdump</code> and printing packets in the terminal.</div>


<div class="alert alert-block alert-info">
<b>Hint:</b> Linux Kernel enables, by default, the Reverse Path Filtering, which tests each incoming packet against the routing table and drops the packet if the <b>source</b> address is not routable through any interface. 
This security feature prevents <a href="https://en.wikipedia.org/wiki/IP_address_spoofing" target="_blank">IP Address Spoofing</a>.</div>




__Optional__: if you are interested, you can read more about it <a href="https://www.theurbanpenguin.com/rp_filter-and-lpic-3-linux-security/" target="_blank">here</a>. 



## Task 1.1 -- NAT Setup


After figuring out the previous problem's cause, you will solve it using [NAT](https://en.wikipedia.org/wiki/Network_address_translation). 
You can use [_nftables_](https://en.wikipedia.org/wiki/Nftables), a subsystem of the Linux kernel that provides packet filtering and NAT.


* In the containers __"router1"__ and __"router2"__, activate [IP Masquerading](https://tldp.org/HOWTO/IP-Masquerade-HOWTO/ipmasq-background2.5.html) to enable NAT/Masquerading for the private networks "private_net1" and "private_net2":


<div style="background: #f8f8f8; overflow:auto;width:auto;border:solid gray;border-width:.1em .1em .1em .8em;padding:.2em .6em;"><pre style="margin: 0; line-height: 125%;color: black;">
sudo nft add table ip nat
sudo nft 'add chain ip nat POSTROUTING { type nat hook postrouting priority 100; }'
sudo nft add rule ip nat POSTROUTING oifname "ether1" masquerade
</pre></div>


<div class="alert alert-block alert-info">
<b>Note:</b> You will use nftables in the next lab to configure firewall rules and packet filtering. 
Thus, it can be helpful to read more about nftables, e.g., <a href="https://wiki.nftables.org/wiki-nftables/index.php/Main_Page" target="_blank">[HOWTO documentation page]</a>.</div>

* Validate the NAT setup by ensuring that you can connect to the webserver from __"pclient1"__ and __"pclient2"__:


<div style="background: #f8f8f8; overflow:auto;width:auto;border:solid gray;border-width:.1em .1em .1em .8em;padding:.2em .6em;"><pre style="margin: 0; line-height: 125%;color: black;">
ping 129.100.1.3
wget http://129.100.1.3 --header "Host: www.ttm4200.com"
</pre></div>

* Start a packet capture in GNS3 on the link connected to __"pclient1"__ and on the link between __"webserver"__ and Switch. Alternatively, similarly to the previous labs, use <code>tcpdump</code> inside __"pclient1"__ and __"webserver"__ containers.

  <div class="alert alert-block alert-info">
  <b>Tip:</b> With the Right-click on the link between "router1" and "pclient1," as well as the link between "webserver" and Switch and select "Start capture" initiate a Wireshark packet capture in GNS3. You can stop the capture in the same manner.
  </div>

* In the __"pclient1"__ container, retrieve the content from the webserver using:

<div style="background: #f8f8f8; overflow:auto;width:auto;border:solid gray;border-width:.1em .1em .1em .8em;padding:.2em .6em;"><pre style="margin: 0; line-height: 125%;color: black;">
wget http://129.100.1.3 --header "Host: www.ttm4200.com"
</pre></div>


* Stop packet capturing at __"pclient1"__ and __"webserver"__.

* Save the capture to files "pclient1.pcap" and "webserver.pcap", respectively.

  <div class="alert alert-block alert-info">
  <b>Tip:</b> Make sure your Wireshark files generated through GNS3 in a correct file format (i.e. ".pcap").
  </div>

* Copy the capture files to your "ntnu_server" inside the "~/labs/lab7" directory.

In [ ]:
check_progress.test_1_1()

## Task 1.2 -- Analyzing Packets From The Client Perspective (Behind NAT)

Open the "pclient1.pcap" file in Wireshark. Apply the display filter `http`to show only HTTP packets, then answer the following questions:

* Find the HTTP GET sent from the client to the webserver. 
What are the source and destination IP addresses and TCP source and destination ports on the IP datagram carrying this HTTP GET?


* Find the corresponding 200 OK HTTP message received from the webserver. 
What are the source and destination IP addresses and TCP source and destination ports on the IP datagram carrying this HTTP 200 OK message?

In [ ]:
# Find the HTTP GET sent from the client to the webserver. 
#What are the source and destination IP addresses and TCP source and destination ports on the IP datagram carrying this HTTP GET?
HTTP_GET_packet_number_behind_nat =  #Your answer (an integer)
HTTP_GET_source_IP_behind_nat =  #You answer (in a string)
HTTP_GET_source_port_behind_nat =  #Your answer (an integer)
HTTP_GET_destination_IP_behind_nat =  #You answer (in a string)
HTTP_GET_destination_port_behind_nat =   #Your answer (an integer)

# Find the corresponding 200 OK HTTP message received from the webserver. 
# What are the source and destination IP addresses and TCP source and destination ports on the IP datagram carrying this HTTP 200 OK message?
HTTP_response_packet_number_behind_nat =  #Your answer (an integer)
HTTP_response_source_IP_behind_nat =  #You answer (in a string)
HTTP_response_source_port_behind_nat =  #Your answer (an integer)
HTTP_response_destination_IP_behind_nat =  #You answer (in a string)
HTTP_response_destination_port_behind_nat =  #Your answer (an integer)

check_progress.test_1_2(HTTP_GET_packet_number_behind_nat,
                        HTTP_GET_source_IP_behind_nat,
                        HTTP_GET_source_port_behind_nat,
                        HTTP_GET_destination_IP_behind_nat,
                        HTTP_GET_destination_port_behind_nat,
                        HTTP_response_packet_number_behind_nat,
                        HTTP_response_source_IP_behind_nat,
                        HTTP_response_source_port_behind_nat,
                        HTTP_response_destination_IP_behind_nat,
                        HTTP_response_destination_port_behind_nat)

## Task 1.3 -- Analyzing Packets From The Webserver Perspective (After NAT)



Open the "webserver.pcap" file in Wireshark, apply the display filter `http`. Then answer these questions:

* Find the HTTP GET message sent from the client to the webserver. What are the source and destination IP addresses and TCP source and destination ports on the IP datagram carrying this HTTP GET?

<div class="alert alert-block alert-info">
    <b>Self-Check:</b> Which of these fields are the same, and which are different, compared to HTTP GET behind NAT? Why?
</div>

* Compared to HTTP GET behind NAT, Which of the following fields in the IP datagram carrying the HTTP GET are changed: Version, Header Length, Flags, Checksum?

<span style="font-size: 25px;">Q2. </span> **REPORT:** Briefly explain why (Version, Header Length, Flags, or Checksum) field has changed. Give two reasons.

* Find the 200 OK HTTP message. What are the source and destination IP addresses and TCP source and destination ports on the IP datagram carrying this HTTP 200 OK message?

<div class="alert alert-block alert-info">
    <b>Self-Check:</b> Which of these fields are the same, and which are different, compared to the 200 OK HTTP message behind NAT? Why?
</div>

In [ ]:
# Find the HTTP GET message sent from the client to the webserver. 
# What are the source and destination IP addresses and TCP source and destination ports on the IP datagram carrying this HTTP GET?
HTTP_GET_packet_number_after_nat =  #Your answer (an integer)
HTTP_GET_source_IP_after_nat =  #You answer (in a string)
HTTP_GET_source_port_after_nat =  #Your answer (an integer)
HTTP_GET_destination_IP_after_nat =  #You answer (in a string)
HTTP_GET_destination_port_after_nat =  #Your answer (an integer)

#Find the 200 OK HTTP message. What are the source and destination IP addresses and TCP source and 
#destination ports on the IP datagram carrying this HTTP 200 OK message?
HTTP_response_packet_number_after_nat =  #Your answer (an integer)
HTTP_response_source_IP_after_nat =  #You answer (in a string)
HTTP_response_source_port_after_nat =  #Your answer (an integer)
HTTP_response_destination_IP_after_nat =  #You answer (in a string)
HTTP_response_destination_port_after_nat =  #Your answer (an integer)

check_progress.test_1_3(HTTP_GET_packet_number_after_nat,
                        HTTP_GET_source_IP_after_nat,
                        HTTP_GET_source_port_after_nat,
                        HTTP_GET_destination_IP_after_nat,
                        HTTP_GET_destination_port_after_nat,
                        HTTP_response_packet_number_after_nat,
                        HTTP_response_source_IP_after_nat,
                        HTTP_response_source_port_after_nat,
                        HTTP_response_destination_IP_after_nat,
                        HTTP_response_destination_port_after_nat)

# Milestone 2 -- Dynamic Host Configuration Protocol (DHCP)

## Task 2.1 -- DHCP Setup

In this section, you will set up a DHCP server and a client.
We will use [ISC DHCP](https://www.isc.org/dhcp/), which offers an open-source DHCP server and client implementation. 
You will set up an authoritative DHCP server for the local network "private_net2" (see figure 1).
The server-side, _isc-dhcp-server_ is already installed in __"router2"__, but you need to configure it.
You can read this [article](https://en.wikiversity.org/wiki/Configure_ISC-DHCP_server) on configuring an ISC-DHCP server.

* Configure the DHCP server's properties in the "/etc/dhcp/dhcpd.conf" file. 
You can use the following template, complete the missing parts that are marked with `===fill_here===`:

```r
#declare this DHCP server as authoritative
===fill_here===;

#specify a subnet and its netmask
subnet ===fill_here=== netmask ===fill_here=== {

   #specify a range of addresses the server can offer
   #(exclude the router IP from this range)
   range ===fill_here=== ===fill_here=== ;

   option domain-name-servers 129.100.1.2; 
   option domain-name "ttm4200.com";
   option routers 10.20.50.100;
}
default-lease-time 600;
max-lease-time 7200;
```

<div class="alert alert-block alert-info">
<b>Tip:</b> In the sample configuration available in the ISC-DHCP server configuration file, you can uncomment the authoritative directive command and edit the "configuration for an internal subnet" and edit it to match the template. You can also comment (with #) all the uncommented lines found in the sample configuration. </div>


* Activate DHCP on the interface "ether0" by editing the "/etc/default/isc-dhcp-server" configuration file (comment "INTERFACESv6").

```r
INTERFACESv4="ether0"
#INTERFACESv6=""
```

* Check the syntax of the server configuration file for any errors: 

<div style="background: #f8f8f8; overflow:auto;width:auto;border:solid gray;border-width:.1em .1em .1em .8em;padding:.2em .6em;"><pre style="margin: 0; line-height: 125%;color: black;">
dhcpd -t
</pre></div>


* Restart the dhcp server:


<div style="background: #f8f8f8; overflow:auto;width:auto;border:solid gray;border-width:.1em .1em .1em .8em;padding:.2em .6em;"><pre style="margin: 0; line-height: 125%;color: black;">
sudo service isc-dhcp-server restart
</pre></div>

* Start packet capturing on the link connected to __"ether0"__ on __"router2"__.

* On the client-side, [_dhclient_](https://linux.die.net/man/8/dhclient) is already installed in __"pclient2"__. 
You will use this client for retrieving a DHCP lease.

* In __"pclient2"__, release any existing DHCP client lease:


<div style="background: #f8f8f8; overflow:auto;width:auto;border:solid gray;border-width:.1em .1em .1em .8em;padding:.2em .6em;"><pre style="margin: 0; line-height: 125%;color: black;">
sudo dhclient -r -v
</pre></div>


* Request a new IP address from a DHCP server:


<div style="background: #f8f8f8; overflow:auto;width:auto;border:solid gray;border-width:.1em .1em .1em .8em;padding:.2em .6em;"><pre style="margin: 0; line-height: 125%;color: black;">
sudo dhclient -v
</pre></div>

* Then, __again__, release the lease.


<div class="alert alert-block alert-warning">
<b>Warning:</b> Ignore, if present, the error message "System has not been booted with systemd as init system (PID 1). Can't operate.". 
This message is systemd-related because we are running Linux in a container, <a href="https://stackoverflow.com/questions/39169403/systemd-and-systemctl-within-ubuntu-docker-images#answer-39169889" target="_blank">for more information</a>. </div>


* Stop the packet capture, save it as "dhcp.pcap" and copy the capture file to your "ntnu_server", inside the ~/labs/lab7 folder.


## Task 2.2 -- Analyzing DHCP Packets

Open the capture file "dhcp.pcap" in Wireshark and display only DHCP frames (apply the display filter `dhcp` ). Then answer the following questions:

* What is the packet number (packet ordinals in the capture file) of the four DHCP messages (Discover, Offer, Request, and ACK)?

* Are DHCP messages sent over UDP or TCP? 

* What is the value of the Transaction-ID in each of the four (Discover/Offer/Request/ACK) DHCP messages?  
 

* What are the source and destination IP addresses for each of the four DHCP messages (Discover/Offer/Request/ACK DHCP)? 

In [ ]:
# What is the packet number (packet ordinals in the capture file) of the four DHCP messages (Discover, Offer, Request, and ACK)?

DHCP_Discover_packet_number =  #Your answer as integer
DHCP_Offer_packet_number =  #Your answer as integer
DHCP_Request_packet_number =  #Your answer as integer
DHCP_ACK_packet_number =  #Your answer as integer

# Are DHCP messages sent over UDP or TCP? 
DHCP_transport_protocol =  # your answer as a string of "UDP" or "TCP"

# What is the value of the Transaction-ID in each of the four (Discover/Offer/Request/ACK) DHCP messages?
Transaction_ID =  #You answer as an integer (decimal or hexadecimal preceded with 0x)


# What are the source and destination IP addresses for each of the four DHCP messages (Discover/Offer/Request/ACK DHCP)? 
DHCP_Discover_source_IP =  #Your answer as a string of IP address
DHCP_Discover_destination_IP =  #Your answer as a string of IP address
DHCP_Offer_source_IP =  #Your answer as a string of IP address
DHCP_Offer_destination_IP =  #Your answer as a string of IP address
DHCP_Request_source_IP =  #Your answer as a string of IP address
DHCP_Request_destination_IP =  #Your answer as a string of IP address
DHCP_ACK_source_IP =  #Your answer as a string of IP address
DHCP_ACK_destination_IP =  #Your answer as a string of IP address


# What is the router and subnetmask in the DHCP offer message? 
DHCP_Offer_router =  #Your answer as a string of IP address
DHCP_Offer_subnetmask =  #Your answer as a string of subnet mask

# How long is the IP address lease time (in seconds)?
IP_lease_time =  #Your answer as integer 

# Does the DHCP server acknowledge the DHCP release message?
server_acknowledge_release_message =  # Your answer as a boolean (True or False)

check_progress.test_2_1(DHCP_Discover_packet_number,
                        DHCP_Offer_packet_number,
                        DHCP_Request_packet_number,
                        DHCP_ACK_packet_number,
                        DHCP_transport_protocol,
                        Transaction_ID,
                        DHCP_Discover_source_IP,
                        DHCP_Discover_destination_IP,
                        DHCP_Offer_source_IP,
                        DHCP_Offer_destination_IP,
                        DHCP_Request_source_IP,
                        DHCP_Request_destination_IP,
                        DHCP_ACK_source_IP,
                        DHCP_ACK_destination_IP,
                        DHCP_Offer_router,
                        DHCP_Offer_subnetmask,
                        IP_lease_time,
                        server_acknowledge_release_message)

<span style="font-size: 25px;">Q3. </span> **REPORT:** Based on your reading of the textbook (pages 371--374), are the source and destination IP addresses of the four DHCP messages what you expected? Briefly explain why?

* What is the router and subnetmask in the DHCP offer message? 

* How long is the IP address lease time?

* Does the DHCP server acknowledge the DHCP release message?